In [ ]:
# Bibliotecas para a análise dos abstracts

!pip install pandas nltk wordcloud matplotlib yake

In [ ]:
# Importando as bibliotecas

import pandas as pd
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Fazendo download dos dados

!wget https://raw.githubusercontent.com/brasileiras-pln/analise-acl-2023/main/abstracts_acl2023.csv

In [ ]:
# Convertendo os dados em uma planilha do Pandas

data_file = "abstracts_acl2023.csv"
df = pd.read_csv(data_file, header=None, names=['Abstract'], delimiter=";")

In [ ]:
# Removendo os trabalhos que não possuem abstract

df['Abstract'].fillna("", inplace=True)
df = df[df['Abstract'] != ""]

In [ ]:
# Quantidade de trabalhos com abstract presente:

trabalhos_abstract = df.shape[0]
print(f"Trabalhos com abstract: {trabalhos_abstract}")

In [ ]:
# Filtrando os trabalhos que possuem o termo "bias"

df = df[df['Abstract'].str.contains("bias")]

In [ ]:
# Quantidade de trabalhos que possuem o termo "bias"

trabalhos_bias = df.shape[0]
print(f"Trabalhos que possuem o termo 'bias': {trabalhos_bias}")

In [ ]:
# Porcentagem de trabalhos que possuem o termo "bias"

print(f"Porcentagem de trabalhos que possuem o termo 'bias': {(trabalhos_bias/trabalhos_abstract*100):.1f}%")

In [ ]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Tokenização e remoção de stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, str):
        words = nltk.word_tokenize(text)
        words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
        return ' '.join(words)
    else:
        return ''

df['Processed_Abstract'] = df['Abstract'].apply(preprocess_text)

In [ ]:
print(df.head())

In [ ]:
# Criando uma wordcloud dos principais termos

all_text = ' '.join(df['Processed_Abstract'])

wordcloud = WordCloud(width=800, height=400, background_color='white', colormap="twilight_shifted_r").generate(all_text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
wordcloud.to_file("word_cloud.png")




# Análise de Unigramas/Bigramas/Trigramas

In [ ]:
import cufflinks as cf

# Criando função para gerar os top N n-gramas

def get_top_n_gram(corpus, n=None, r=None):
    vec = CountVectorizer(ngram_range=(r, r), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
# Analisando as top-20 palavras (unigramas)

common_words = get_top_n_gram(df['Processed_Abstract'], 20, 1)

for word, freq in common_words:
    print(f"{word} ({freq})")

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

df4 = pd.DataFrame(common_words, columns = ['Processed_Abstract' , 'count'])
df4.groupby('Processed_Abstract').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', color='red', yTitle='Quantidade', linecolor='black', title='Top 20 unigramas', asFigure=True).show(renderer="colab")


In [ ]:
# Analisando os top-20 conjuntos de 2 palavras (bigramas)

common_words = get_top_n_gram(df['Processed_Abstract'], 20, 2)

for word, freq in common_words:
    print(f"{word} ({freq})")

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

df4 = pd.DataFrame(common_words, columns = ['Processed_Abstract' , 'count'])
df4.groupby('Processed_Abstract').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', color='blue', yTitle='Quantidade', linecolor='black', title='Top 20 bigramas', asFigure=True).show(renderer="colab")


In [ ]:
# Analisando os top-20 conjuntos de 3 palavras (trigramas)

common_words = get_top_n_gram(df['Processed_Abstract'], 20, 3)

for word, freq in common_words:
    print(f"{word} ({freq})")

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

df4 = pd.DataFrame(common_words, columns = ['Processed_Abstract' , 'count'])
df4.groupby('Processed_Abstract').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Quantidade', linecolor='black', title='Top 20 trigramas', asFigure=True).show(renderer="colab")


# LDA Analysis

In [ ]:
# Análise utilizando modelagem de tópicos
#https://blog.devgenius.io/working-with-pyldavis-topic-modeling-exploration-tool-b03682d57079

!pip install pyldavis
!pip install pandas==1.5.1

In [ ]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from gensim import matutils
from gensim.corpora import Dictionary
import pyLDAvis
import pyLDAvis.gensim
from pprint import pprint

# Criando um vetorizador de bag-of-words
# Cortamos palavras de baixa e alta frequência e filtramos stopwords em inglês

vectorizer = CountVectorizer(min_df=10, max_df=0.5, stop_words="english")
dtm = vectorizer.fit_transform(df['Processed_Abstract'])

def tokenize(text):
    return nltk.word_tokenize(text)

# Aplicando a função de tokenização a cada resumo
df['Tokenized_Abstract'] = df['Processed_Abstract'].apply(tokenize)

# Criando um dicionário
id2word = Dictionary(df['Tokenized_Abstract'])

# Criando um corpus
corpus = [id2word.doc2bow(tokens) for tokens in df['Tokenized_Abstract']]

In [ ]:
from gensim.models.ldamodel import LdaModel

# Construindo LDA model

num_topics = 5

lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=num_topics,
                   iterations=50000,
                   random_state=100,
                   update_every=1,
                   eval_every=10,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

In [ ]:
# Exibindo os principais termos de cada tópico e seu score

pprint(lda_model.print_topics(num_words=30))
doc_lda = lda_model[corpus]

In [ ]:
# Criando visualização de distância de tópicos

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)